In [19]:
from napkin import response, request
from pymongo import MongoClient
from datetime import datetime
import json
import secrets
import os

CLIENT_NAME = request.path_params.get("CLIENT_NAME")
MONGO_AUTH = os.environ.get("MONGO_AUTH")
body = json.loads(request.data)

In [3]:
from passlib.context import CryptContext

pwd_context = CryptContext(
  schemes=["pbkdf2_sha256"],
  default="pbkdf2_sha256",
  pbkdf2_sha256__default_rounds=30000
)

In [4]:
client = MongoClient("mongodb+srv://{}@meusite.hcxhp.mongodb.net/Site?retryWrites=true&w=majority".format(MONGO_AUTH))
db = client.Site

Coll = db[CLIENT_NAME]
Auth = Coll.find_one({ "_id": "Auth", "users.email": body["email"] })

In [18]:
if Auth is None:
  response.status_code = 401

users = Auth.get("users")
index, user = [(i, usr) for (i, usr) in enumerate(users) if usr.get("email") == body["email"]][0]

{'email': 'leonardogviva@gmail.com',
 'pass': '$pbkdf2-sha256$30000$RIiRkjJG6J1zTilljFFKyQ$gIdJjCHSDJMg63u2bidgsaZoec34G0SzQJBZ4yz7bX8'}

In [28]:
password_match = pwd_context.verify(body["pass"], user.get("pass"))

if password_match:
  login_hash = secrets.token_hex()
  user["service"] = {
    "token": login_hash,
    "since": datetime.now()
  }
  users[index] = user

  Coll.update_one({ "_id": "Auth" }, { "$set": { "users": users } })

  response.body = login_hash
  response.status_code = 200
else:
  response.status_code = 401
